In [1]:
import numpy as np
import tensorflow as tf
import utils
import encoder as enc
import dataset as ds
import train as tr
import decoder as dec
import dataset


Using TensorFlow backend.


In [3]:
#======= FLAGS ==========
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('maxout_layer_size', 100, 'Maxout layer size')
tf.app.flags.DEFINE_integer('max_sequence_length', 160, 'Max length of context')
tf.app.flags.DEFINE_integer('max_question_length', 40, 'Max question tokens length')
tf.app.flags.DEFINE_float('learning_rate', 0.005, 'Learning Rate')
tf.app.flags.DEFINE_integer('maxout_pooling_size', 8, 'Maxout pooling size')
tf.app.flags.DEFINE_integer('lstm_size', 100, 'LSTM cell internal size')
tf.app.flags.DEFINE_string('log_path', '/tmp/working/logs', 'logs location')
tf.app.flags.DEFINE_integer('acc_batch_size', 10, 'How many examples to use to calculate accuracy')
#tf.app.flags.DEFINE_integer('train_batch_size', 20, 'Train Batch Size')
tf.app.flags.DEFINE_integer('max_decoder_iterations', 4, 'Decoder Iterations')
tf.app.flags.DEFINE_integer('max_epoch', 100, 'Max Train Epoch Count')


ArgumentError: argument --maxout_layer_size: conflicting option string: --maxout_layer_size

In [4]:
# remove all variables
#tf.reset_default_graph();

lstm_size = FLAGS.lstm_size
acc_batch_size = FLAGS.acc_batch_size
word_vector_size = 300
maxout_pooling_size = FLAGS.maxout_pooling_size
max_decoder_iterations = FLAGS.max_decoder_iterations
maxout_layer_size = FLAGS.maxout_layer_size;
max_epoch = FLAGS.max_epoch;
max_sequence_length = FLAGS.max_sequence_length;
max_question_length = FLAGS.max_question_length


batch_size = tf.placeholder(tf.int32, ())


dropout_rate_ph = tf.placeholder(tf.float32)
question_ph = tf.placeholder(tf.float32, [None, max_question_length, word_vector_size], name="q_input")
document_ph = tf.placeholder(tf.float32, [None, max_sequence_length, word_vector_size], name="d_input")
doc_len_ph = tf.placeholder(tf.int32, [None])
que_len_ph = tf.placeholder(tf.int32, [None])
start_true = tf.placeholder(tf.int32, [None]);
end_true   = tf.placeholder(tf.int32, [None]);
document_size = doc_len_ph
question_size = que_len_ph

with tf.name_scope('ENCODER'):
    # LSTM cell initialization
    lstm = tf.nn.rnn_cell.LSTMCell(lstm_size)
    lstm = tf.nn.rnn_cell.DropoutWrapper(cell=lstm, output_keep_prob=dropout_rate_ph)


# LSTM cells for Bi-LSTM for COATINATION ENCODER
with tf.name_scope('COATTENTION_ENCODER'):
    lstm_cenc_fw = tf.nn.rnn_cell.LSTMCell(lstm_size)
    lstm_cenc_fw = tf.nn.rnn_cell.DropoutWrapper(cell=lstm_cenc_fw, output_keep_prob=dropout_rate_ph)
    lstm_cenc_bw = tf.nn.rnn_cell.LSTMCell(lstm_size)
    lstm_cenc_bw = tf.nn.rnn_cell.DropoutWrapper(cell=lstm_cenc_bw, output_keep_prob=dropout_rate_ph)

# create lstm cell for DYNAMIC POINTING DECODER
lstm_dec = tf.contrib.rnn.BasicLSTMCell(lstm_size)
# get lstm initial state of zeroes
#lstm_dec_state = lstm_dec.zero_state(1, tf.float32)
#start_pos = 0; # ?generate random between (0, document_size-1)
#end_pos = 0;   # ?generate random between (0, document_size-1)

# create sentinel vector variable for both encodings 
#with tf.variable_scope("scope1") as scope:
sentinel_q = tf.get_variable("sentinel_q", [ 1, lstm_size ], initializer = tf.random_normal_initializer())
sentinel_d = tf.get_variable("sentinel_d", [ 1, lstm_size ], initializer = tf.random_normal_initializer()) 

tf.summary.histogram('sentinel_q', sentinel_q)
tf.summary.histogram('sentinel_q_max', tf.reduce_max(sentinel_q))
tf.summary.histogram('sentinel_d', sentinel_d)
tf.summary.histogram('sentinel_d_max', tf.reduce_max(sentinel_d))

# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

In [5]:
# (batch, D, 2L)
U = enc.encoderBatch(
    document_ph, question_ph, 
    document_size, question_size, 
    lstm, lstm_cenc_fw, lstm_cenc_bw, 
    sentinel_d, sentinel_q,
    batch_size,
    FLAGS)
print(U)

Tensor("COATTENTION_ENCODER_1/Slice_2:0", shape=(?, ?, ?), dtype=float32)


In [6]:
# ===================== DYNAMIC POINTING DECODER =============

sum_start_scores, sum_end_scores = dec.decoderBatch(U, lstm_dec, dropout_rate_ph, batch_size, FLAGS)
sum_loss, accuracy_avg, pr_start_idx, pr_end_idx = tr.loss_and_accuracy(start_true, end_true, batch_size, sum_start_scores, sum_end_scores, max_sequence_length)

tf.summary.histogram('sum_start_scores', sum_start_scores)
tf.summary.histogram('sum_end_scores', sum_end_scores)
tf.summary.scalar('loss_test', sum_loss, ["TEST_STAT"])
tf.summary.scalar('loss_train', sum_loss, ["TRAIN_STAT"])
tf.summary.scalar('avg_accuracy_test',  accuracy_avg, ["TEST_STAT"])
tf.summary.scalar('avg_accuracy_train', accuracy_avg, ["TRAIN_STAT"])


with tf.name_scope('Train'):
    train_step = optimizer.minimize(sum_loss)


In [7]:
#=========== Training ==================

summary_op_train = tf.summary.merge_all("TRAIN_STAT")
summary_op_test = tf.summary.merge_all("TEST_STAT")
summary_op = tf.summary.merge_all()

embeddings = dataset.Embeddings('./english/glove.840B.300d.w2vformat.bin')
#embeddings = dataset.Embeddings('./processed_ruscorpora_1_300_10.bin')
init = tf.global_variables_initializer()
DATASET_LENGTH = 20
BATCH_SIZE = 5

with tf.Session() as sess:
    sess.run(init)
    h_param_str = utils.make_h_param_string(FLAGS.learning_rate, FLAGS.lstm_size, max_sequence_length, FLAGS.maxout_pooling_size)
    writer = tf.summary.FileWriter(FLAGS.log_path + "/1-" + h_param_str, sess.graph)
    with open('./english/test_160.csv') as file_test:
        for epoch_ in range(100 or max_epoch):
            step_ = 0;
            with open('./english/train_160.csv') as file_train:
                while True:
                    feed_dict = tr.processLineBatch(file_train, embeddings, BATCH_SIZE, 
                                                    max_sequence_length, max_question_length, 
                                                    question_ph, document_ph, dropout_rate_ph,
                                                    doc_len_ph, que_len_ph, start_true, end_true,
                                                    batch_size,
                                                    0.5)
                    if feed_dict is None: break
                    tr.trainStep(sess, 
                                 feed_dict, 
                                 writer, 
                                 train_step, sum_loss, accuracy_avg, summary_op, summary_op_train, 
                                 epoch_ * DATASET_LENGTH + step_, profiling=False)
                    step_+= 1
                    if (step_ >= DATASET_LENGTH): break;
                        
            print('Epoch', epoch_, 'completed')
            test_params = tr.processLineBatch(file_test, embeddings, BATCH_SIZE, 
                                              max_sequence_length, max_question_length, 
                                              question_ph, document_ph, dropout_rate_ph,
                                              doc_len_ph, que_len_ph, start_true, end_true,
                                              batch_size,
                                              1)
            tr.accuracyTest(sess, test_params, writer, 
                            accuracy_avg, summary_op, summary_op_test, pr_start_idx, pr_end_idx, epoch_)
        print('End')
            

Embeddings are loaded to memory
Epoch 0 completed
AVG accuracy 0.0
Epoch 1 completed
AVG accuracy 0.1
Epoch 2 completed
AVG accuracy 0.0
Epoch 3 completed
AVG accuracy 0.2
Epoch 4 completed
AVG accuracy 0.0
Epoch 5 completed
AVG accuracy 0.0410256410256
Epoch 6 completed
AVG accuracy 0.0
Epoch 7 completed
AVG accuracy 0.0
Epoch 8 completed
AVG accuracy 0.0
Epoch 9 completed
AVG accuracy 0.0
Epoch 10 completed
AVG accuracy 0.0
Epoch 11 completed
AVG accuracy 0.2
Epoch 12 completed
AVG accuracy 0.0
Epoch 13 completed
AVG accuracy 0.2
Epoch 14 completed
AVG accuracy 0.2
Epoch 15 completed
AVG accuracy 0.0
Epoch 16 completed
AVG accuracy 0.0
Epoch 17 completed
AVG accuracy 0.0
Epoch 18 completed
AVG accuracy 0.0
Epoch 19 completed
AVG accuracy 0.0
Epoch 20 completed
AVG accuracy 0.333333333333
Epoch 21 completed
AVG accuracy 0.2
Epoch 22 completed
AVG accuracy 0.18
Epoch 23 completed
AVG accuracy 0.0
Epoch 24 completed
AVG accuracy 0.0
Epoch 25 completed
AVG accuracy 0.1
Epoch 26 completed